In [ ]:
import socketserver
import threading
import os # os.path.getsize(path)
import time
 
HOST = ''
PORT = 9100
lock = threading.Lock() # syncronized 동기화 진행하는 스레드 생성

class UserManager: # 사용자관리 및 채팅 메세지 전송을 담당하는 클래스
                   # ① 채팅 서버로 입장한 사용자의 등록
                   # ② 채팅을 종료하는 사용자의 퇴장 관리
                   # ③ 사용자가 입장하고 퇴장하는 관리
                   # ④ 사용자가 입력한 메세지를 채팅 서버에 접속한 모두에게 전송
 
    def __init__(self):
        self.users = {} # 사용자의 등록 정보를 담을 사전 {사용자 이름:(소켓,주소),...}
 
    def addUser(self, username, conn, addr): # 사용자 ID를 self.users에 추가하는 함수
        if username in self.users: # 이미 등록된 사용자라면
            conn.send('이미 등록된 사용자입니다.\n'.encode())
            return None
 
              # 새로운 사용자를 등록함
        lock.acquire() # 스레드 동기화를 막기위한 락
        self.users[username] = (conn, addr)
        lock.release() # 업데이트 후 락 해제

        self.sendMessageToAll('system@normal_chatting@[%s]님이 입장했습니다. \n' %username)
        print('+++ 대화 참여자 수 [%d]' %len(self.users))

        return username
 
    def removeUser(self, username): #사용자를 제거하는 함수
        if username not in self.users:
            return

        lock.acquire()
        del self.users[username]
        lock.release()
 
        self.sendMessageToAll('[%s]님이 퇴장했습니다.' %username)
        print('--- 대화 참여자 수 [%d]' %len(self.users))
 
    def messageHandler(self, username, msg): # 전송한 msg를 처리하는 부분
        
        if msg.split('@')[1] == 'image_send_server_to_client':
            print('파일 전송 요청을 받았습니다 server -> client')
            print(msg)
            file_name = msg.split('@')[2].strip()
            print('파일 이름 : ' + file_name)
            file_size = str(os.path.getsize(file_name))
            print('파일 길이 : ' + file_size)
            #if not exists(file_name): # 파일이 해당 디렉터리에 존재하지 않으면
            #    return # handle()함수를 빠져 나온다.
            self.sendMessageToAll('system@image_send_server_to_client@stoc_image.jpg@' + file_size + "\n")
            data_transferred = 0
            with open(file_name, 'rb') as f:
                try:
                    data = f.read(1024) # 파일을 1024바이트 읽음
                    while True: # 파일이 빈 문자열일때까지 반복
                        self.sendFileToAll(data)
                        data_transferred += len(data)
                        #print(len(data))
                        if data_transferred >= int(file_size):
                            break
                        time.sleep(0.01)
                        data = f.read(1024)
                except Exception as e:
                    print(e)
                f.flush()
                f.close()
            print('전송완료[%s], 전송량[%d]' %(file_name,data_transferred))
            
        else: #msg[0] != '/': # 보낸 메세지의 첫문자가 '/'가 아니면
            self.sendMessageToAll('%s' %(msg))
            return
 
        #if msg.strip() == '/quit': # 보낸 메세지가 'quit'이면
        #    self.removeUser(username)
        #    return -1
 
    def sendMessageToAll(self, msg):
        for conn, addr in self.users.values():
            print('뿌린 메시지 : ' + msg ,end='')
            conn.sendall(msg.encode())
    
    def sendFileToAll(self, msg):
        for conn, addr in self.users.values():
            conn.sendall(msg)
            
           
 
class MyTcpHandler(socketserver.BaseRequestHandler):
    userman = UserManager()
     
    def handle(self): # 클라이언트가 접속시 클라이언트 주소 출력
        print('[%s] 연결됨' %self.client_address[0])
 
        try:
            username = self.registerUsername()
            msg = self.request.recv(1024)
            while msg:
                msg = msg.decode()
                if msg.split('@')[1] == 'image_send_client_to_server':
                    print('파일 전송 요청을 받았습니다 client -> server', end='')
                    print(msg)
                    file_name = msg.split('@')[2].strip()
                    file_length_str = msg.split('@')[3].strip()
                    file_length_int = int(file_length_str)
                    print('파일 이름 : ' + file_name)
                    print('파일 길이 : ' + file_length_str)
                    file_size_count = 0
                   
                    with open(file_name, 'wb') as f:
                        try:
                            data = self.request.recv(1)
                            while data:
                                #print('수신중 : ' + str(data))
                                
                                f.write(data)
                                file_size_count += len(data)
                                if(file_size_count  >= file_length_int):
                                    break
                                data = self.request.recv(1)
                            print('파일 수신 완료 : ' + str(file_size_count))
                            #data = self.request.recv(1024)
                        except Exception as e:
                            print(e)
                    #####
                    
                    
            
                    #####       
                elif msg.split('@')[1] == 'image_send_client_to_server_hangeul':
                    print('한글 인식 요청을 받았습니다 client -> server', end='')
                    print(msg)
                    file_name = msg.split('@')[2].strip()
                    file_length_str = msg.split('@')[3].strip()
                    file_length_int = int(file_length_str)
                    print('파일 이름 : ' + file_name)
                    print('파일 길이 : ' + file_length_str)
                    file_size_count = 0
                   
                    with open(file_name, 'wb') as f:
                        try:
                            data = self.request.recv(1024)
                            while data:
                                #print('수신중 : ' + str(data))
                                
                                f.write(data)
                                file_size_count += len(data)
                                if(file_size_count  >= file_length_int):
                                    break
                                data = self.request.recv(1024)
                            print('파일 수신 완료 : ' + str(file_size_count))
                            #data = self.request.recv(1024)
                        except Exception as e:
                            print(e)

                else: 
                    if self.userman.messageHandler(username, msg) == -1:
                        self.request.close()
                        break
                msg = self.request.recv(1024)
                 
        except Exception as e:
            print(e)
 
        print('[%s] 접속종료' %self.client_address[0])
        self.userman.removeUser(username)
 
    def registerUsername(self):
        while True:
            username = self.request.recv(1024)
            username = username.decode().strip()
            if self.userman.addUser(username, self.request, self.client_address):
                return username
 

class ChatingServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
        pass
         
def runServer():
    print('+++ 채팅 서버를 시작합니다.')
    print('+++ 채팅 서버를 끝내려면 Ctrl-C를 누르세요.')
 
    try:
        server = ChatingServer((HOST, PORT), MyTcpHandler)
        server.serve_forever()
    except KeyboardInterrupt:
        print('--- 채팅 서버를 종료합니다.')
        server.shutdown()
        server.server_close()

        
runServer()

+++ 채팅 서버를 시작합니다.
+++ 채팅 서버를 끝내려면 Ctrl-C를 누르세요.
[20.20.2.61] 연결됨
뿌린 메시지 : system@normal_chatting@[555]님이 입장했습니다. 
+++ 대화 참여자 수 [1]
뿌린 메시지 : 555@normal_chatting@ㅁㅁㅁㄹ
뿌린 메시지 : 555@normal_chatting@655541225
파일 전송 요청을 받았습니다 client -> server555@image_send_client_to_server@test_image.jpg@5120

파일 이름 : test_image.jpg
파일 길이 : 5120
파일 수신 완료 : 5120
파일 전송 요청을 받았습니다 server -> client
555@image_send_server_to_client@test_image.jpg@

파일 이름 : test_image.jpg
파일 길이 : 5120
뿌린 메시지 : system@image_send_server_to_client@stoc_image.jpg@5120
전송완료[test_image.jpg], 전송량[5120]
[WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다
[20.20.2.61] 접속종료
--- 대화 참여자 수 [0]
[20.20.2.61] 연결됨
뿌린 메시지 : system@normal_chatting@[86757]님이 입장했습니다. 
+++ 대화 참여자 수 [1]
파일 전송 요청을 받았습니다 client -> server86757@image_send_client_to_server@test_image.jpg@5120

파일 이름 : test_image.jpg
파일 길이 : 5120
파일 수신 완료 : 5120
파일 전송 요청을 받았습니다 server -> client
86757@image_send_server_to_client@test_image.jpg@

파일 이름 : test_image.jpg
파일 길이 : 5120
뿌린 메시지 : system@image

뿌린 메시지 : 8582@normal_chatting@ㅗ아아
[20.20.2.61] 접속종료
--- 대화 참여자 수 [0]
